In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)
import sys
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D,Conv2D,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
#import cPickle as pickle
#from video_process_utils import *
import collections
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0" 

Using TensorFlow backend.


In [2]:
def ImportImage(filepath):
    #print(filepath)
    img = Image.open(filepath)
    return np.array(img)

In [4]:
imagepaths_labels = pd.read_csv('./data/imagepath_gdi_10.csv')
msk = np.random.rand(len(imagepaths_labels)) < 0.8
train=imagepaths_labels[msk]
validation=imagepaths_labels[~msk]

In [5]:
train_imgs = np.array([ImportImage(img) for img in train['image_path'].values])
train_labels = np.array([label for label in train['labels_norm'].values])
validation_imgs = np.array([ImportImage(img) for img in validation['image_path'].values])
validation_labels = np.array([label for label in validation['labels_norm'].values])

In [6]:
print('train images shape : ',train_imgs.shape)
print('train labels shape : ',train_labels.shape)
print('validation images shape : ',validation_imgs.shape)
print('validation labels shape : ',validation_labels.shape)

train images shape :  (311, 480, 640, 3)
train labels shape :  (311,)
validation images shape :  (85, 480, 640, 3)
validation labels shape :  (85,)


In [7]:
for i, image in enumerate(train_imgs):
    if (np.count_nonzero(image)<1):
        print('all zeros at : ', i)

In [8]:
for i, image in enumerate(validation_imgs):
    if (np.count_nonzero(image)<1):
        print('all zeros at : ', i)

In [9]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

checkpoint_folder = "./cnn_checkpoints_gdicnn"
batch_size  = 16
num_classes = len(train_labels)
epochs = 10
epochs_drop,drop_factor = (10,0.8)
input_shape = train_imgs[0].shape
kernel_size = 8
conv_dim = 16
initial_lrate = 0.001
dropout_amount = 0.5
l2_lambda = 10**(-3.5)
reg = keras.regularizers.l2(l2_lambda)
last_layer_dim=18

In [10]:
model = Sequential()
model.add(Conv2D(conv_dim, kernel_size=kernel_size, input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(dropout_amount))
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(dropout_amount))
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=3))
#model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}.hdf5"
if train_model:

    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss='mse',metrics=['accuracy'],optimizer=opt)
    #model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])


    checkpoint = \
        ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(train_imgs, train_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(validation_imgs,validation_labels),
              batch_size=batch_size, epochs=epochs,shuffle=True)

Train on 311 samples, validate on 85 samples
Epoch 1/10


In [57]:
batch_size  = 18
dropout_amount = 0.1
checkpoint_folder = "./cnn_checkpoints_gdicnn_loss_sce"
reg = keras.regularizers.l2(l2_lambda)

In [64]:
from keras.utils.np_utils import to_categorical
train_categorical_labels = to_categorical(train_labels, num_classes=None)
validation_categorical_labels = to_categorical(validation_labels, num_classes=None)

In [65]:
type(train_categorical_labels)

numpy.ndarray

In [66]:
len(train_categorical_labels[0])

18

In [67]:
model = Sequential()
model.add(Conv2D(conv_dim, kernel_size=kernel_size, input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=3))
model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(18, activation='softmax'))

In [68]:
train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}.hdf5"
if train_model:

    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    #model.compile(loss=keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'],optimizer=opt)
    model.compile(loss=keras.losses.categorical_crossentropy,metrics=['accuracy'],optimizer=opt)
    #model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])


    checkpoint = \
        ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(train_imgs, train_categorical_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(validation_imgs,validation_categorical_labels),
              batch_size=batch_size, epochs=epochs,shuffle=True)

Train on 3834 samples, validate on 951 samples
Epoch 1/10
3834/3834 [==============================] - 310s 81ms/step - loss: 3.0017 - acc: 0.1403 - val_loss: 2.8217 - val_acc: 0.1314

Epoch 00001: val_loss improved from inf to 2.82170, saving model to ./cnn_checkpoints_gdicnn_loss_sce/weights-01.hdf5
Epoch 2/10
3834/3834 [==============================] - 304s 79ms/step - loss: 2.7486 - acc: 0.1547 - val_loss: 2.7242 - val_acc: 0.1535

Epoch 00002: val_loss improved from 2.82170 to 2.72421, saving model to ./cnn_checkpoints_gdicnn_loss_sce/weights-02.hdf5
Epoch 3/10
3834/3834 [==============================] - 305s 80ms/step - loss: 2.6770 - acc: 0.1638 - val_loss: 3.1110 - val_acc: 0.1493

Epoch 00003: val_loss did not improve from 2.72421
Epoch 4/10
3834/3834 [==============================] - 309s 81ms/step - loss: 2.6561 - acc: 0.1596 - val_loss: 2.6370 - val_acc: 0.1514

Epoch 00004: val_loss improved from 2.72421 to 2.63705, saving model to ./cnn_checkpoints_gdicnn_loss_sce/weig